This notebook is my take on the Kaggle competition on Natural Language Processing (NLP) to predict which Tweets are about real disasters and which one’s aren’t. I'm joining this competition while following a Coursera course on Deep Learning, so I am demanded to use a Recurrent Neural Network as classifier. In order to understand the characteristics of the RNN technique better I will compare it with other NLP approaches (see later on). My code is also available on github: https://github.com/Pdegoffau/kaggle-nlp-disaster

First I will explore the available data in order to be able to form a master plan for achieving a high score in the competition.
# Data exploration

In [2]:
import pandas as pd

df_train = pd.read_csv('input/train.csv')
df_train.sample(10)


,id,keyword,location,text,target
6873,9854,trauma,"Chicago, Illinois",80 @UChicago faculty members pushing universit...,1
2783,4003,disaster,Portoviejo-Manabi-Ecuador,I'm a disaster?? https://t.co/VCV73BUaCZ,0
4459,6342,hostages,Midwest,Sinjar Massacre Yazidis Blast Lack of Action O...,1
7527,10765,wreckage,iTunes,#science Now that a piece of wreckage from fli...,1
2596,3725,destroyed,"ÌÏT: 6.4682,3.18287",Flood: Two people dead 60 houses destroyed in ...,1
1142,1646,bombing,NaN,The cryptic words that guided pilots on the Hi...,1
976,1412,body%20bag,NaN,#handbag #fashion #style http://t.co/iPXpI3me1...,0
3938,5599,flood,New York,Spot Flood Combo 53inch 300W Curved Cree LED W...,0
5849,8360,ruin,followurDREAMS(& my instagram),fresh out da shower lookss ?? (still loving th...,0
5729,8176,rescuers,Washington,#News: 'Many deaths' in shipwreck: Rescuers ar...,1


In [3]:
df_test = pd.read_csv('input/test.csv')
df_test.sample(10)

,id,keyword,location,text
2468,8249,rioting,NaN,#BHRAMABULL Watch Run The Jewels Use Facts to ...
544,1782,buildings%20on%20fire,?arsehole squad?,@blainescronuts thats what i would do if there...
1123,3707,destroyed,Chicago,poor dude...he needs him some valerie...lol RT...
1511,5030,eyewitness,Washington DC,Marked as to-read: DK Eyewitness Travel Guide ...
1106,3651,destroy,Between Manchester and Lille.,@Adz77 You spelled Ronda wrong last night and ...
3196,10619,wounded,Singapore,Suspected Salvadoran gang members killed four ...
1044,3427,derail,"w/ @_ridabot, probably",@PumpkinMari_Bot lemme just derail this real q...
1758,5941,hazard,Mystic Falls,There will be a 3-match ban for any player cau...
2418,8082,rescue,USA,Beauty Deals : http://t.co/eUd317Eptp #4552 Lo...
1897,6401,hurricane,NaN,@al_thegoon i like hurricane habor next to sli...


Great, the training data contains 5 columns, id, keyword, location, text and target. The test file (needed for the final submission only misses the target column, which is as expected. Let's see which columns we can use by exploring them.

In [4]:
df_train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [5]:
df_test.isna().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

In [6]:
df_train.shape

(7613, 5)

In [7]:
df_test.shape

(3263, 4)

So, roughly 1/3 of the entries do not contain a location, which makes this column not suitable as input for a classifier. The keyword is not missing that often, but we have to see if the keyword is meaningful enough to keep as a feature (and drop the rows with missing keywords).

In [8]:
print('Total number of entries={}, Unique ids={}, unique keywords={}, unique locations={}'.format(len(df_train),len(df_train.id.unique()),len(df_train.keyword.unique()),len(df_train.location.unique())))

Total number of entries=7613, Unique ids=7613, unique keywords=222, unique locations=3342


In [9]:
df_train[['keyword','target']].groupby('keyword').value_counts()

keyword     target
ablaze      0         23
            1         13
accident    1         24
            0         11
aftershock  0         34
                      ..
wreck       0         30
            1          7
wreckage    1         39
wrecked     0         36
            1          3
Length: 438, dtype: int64

As we can see using the keywords alone would not lead to accurate results. Therefore, we'll inspect the text field further to see how we can use the information.

In [10]:
df_train['text'].sample(50)

7425    Officer Wounded Suspect Killed in Exchange of ...
471     India shud not give any evidence 2 pak.They wi...
3607    Robert Ballew's log statements are always at t...
1680    Soultech - Collide (Club Mix) http://t.co/8xIx...
2325    'Dangerous' property in downtown Phoenix demol...
3766    @Justin_Ling I promise not to tax pancakes or ...
3139    God forbid anyone in my family knows how to an...
3479    Exploring New Worlds: Three Moments of an Expl...
3533    DK Eyewitness Travel Guide : Chicago by Dorlin...
1143    70th anniversary of Hiroshima atomic bombing m...
5976    @blanksocietyx @emmerdale IM SCREAMING HES MY ...
2850    Elem Pomo helping the displaced from the Rocky...
5166                         Time to obliterate this sin!
2699    2015 new fashion ladies gold watch waterproof ...
77      Traffic accident N CABRILLO HWY/MAGELLAN AV MI...
3203    @chrisroth98 @chaselabsports in an emergency s...
7256    I stand alone\ndon't piss and moan\nabout my c...
2452    Madhya

# Cleaning data

From the text we can extract a few actions we can do to make the text suitable for using it as classification.
- We can remove (or extract) mentions like @abysmaljoiner
- The same applies to hashtags, which we could consider as keywords as well
- Remove special characters like "ÛÏÛ"

Let's start with extracting the mentions and hashtags to see if it can help with classification.

In [11]:
import re
import numpy as np

mention_str = '@([^ ]+)'
hashtag_str = '#([^ ]+)'
special_chars = '[^a-zA-Z]'

def extract_matches(sentence,expr) :
    
    return re.findall(expr,sentence)

df_train['mentions'] = df_train['text'].apply(lambda string: extract_matches(string, mention_str)).apply(lambda y: np.nan if len(y)==0 else y)

df_train['hashtags'] = df_train['text'].apply(lambda string: extract_matches(string, hashtag_str)).apply(lambda y: np.nan if len(y)==0 else y)
df_train.sample(50)

,id,keyword,location,text,target,mentions,hashtags
1413,2039,casualties,NaN,@AlcoholAndMetal + do anything to fix that. Of...,1,[AlcoholAndMetal],NaN
5739,8190,rescuers,NaN,Fears over missing migrants in Med: Rescuers s...,1,NaN,NaN
2049,2941,danger,Instagram: trillrebel_,Guns are for protection.. \nThat shit really s...,0,NaN,NaN
3055,4383,earthquake,NaN,There has not been 1 real tear out of #Shelli ...,0,NaN,"[Shelli, bb17]"
6179,8815,sirens,Sydney,Marketforce Perth named winner of Sirens round...,0,NaN,NaN
1639,2367,collapsed,NaN,1 hour parade like 50 people collapsed. #OneHe...,1,NaN,[OneHeartOneMindOneCSS]
7405,10593,wounded,Venezuela,Richmond police officer wounded suspect killed...,1,NaN,NaN
3021,4336,dust%20storm,CA via Brum,Wall of noise is one thing - but a wall of dus...,1,NaN,NaN
1234,1775,buildings%20on%20fire,"Manchester, NH",Fire destroys two buildings on 2nd Street in #...,1,NaN,[Manchester]
7502,10731,wreck,Canada BC,@raineishida lol...Im just a nervous wreck :P,0,[raineishida],NaN


In [12]:
df_train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
mentions    5595
hashtags    5857
dtype: int64

In [13]:
df_train[df_train['keyword'].isna()]['hashtags'].notna().sum()

21

Unfortunately the number of entries with NaN mentions or hashtags is high, and only 21 keywords can be extracted additionally from the hashtags, which I don't deem high enough to risk added bias. Therefore, we will replace the special characters from the text.

In [14]:

def replace_matches(sentence,expr,replacement) :
    
    return re.sub(expr,replacement,sentence)

# remove mentions
df_train['cleantext'] = df_train['text'].apply(lambda string: replace_matches(string, mention_str, ' ')).apply(lambda y: np.nan if len(y)==0 else y)

# replace special characters
df_train['cleantext'] = df_train['cleantext'].apply(lambda string: replace_matches(string, special_chars, ' ')).apply(lambda y: np.nan if len(y)==0 else y)
df_train[['text','cleantext']].sample(50)

,text,cleantext
2536,This desperation dislocation\nSeparation conde...,This desperation dislocation Separation condem...
5267,@TroySlaby22 slicker than an oil spill,slicker than an oil spill
4813,@TANSTAAFL23 It's not an 'impulse' and it does...,It s not an impulse and it doesn t end in ...
2919,@_dmerida my tears drowned out the terrible ta...,my tears drowned out the terrible taste also...
5178,'But time began at last to obliterate the fres...,But time began at last to obliterate the fres...
256,@CalFreedomMom @steph93065 not to mention a ma...,not to mention a major contributor to the ...
4598,JOBOOZOSO: USAT usatoday_nfl Michael Floyd's h...,JOBOOZOSO USAT usatoday nfl Michael Floyd s h...
6837,Hollywood Movie About Trapped Miners Released ...,Hollywood Movie About Trapped Miners Released ...
1851,My woman crush wedneday goes to the beautiful ...,My woman crush wedneday goes to the beautiful ...
6554,I Survived A 12-Step Program for Sex Addiction...,I Survived A Step Program for Sex Addiction...


This cleaning would maybe be too rigorous, since I see URLs are invalidaded, which leads to new non-existing words. At twitter, URLs are often shortened due to the character limit, so the actual links seem meaningless. Let's therefore replace urls by the word 'url'.

In [15]:
url_regex = '(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])'

# remove mentions
df_train['cleantext'] = df_train['text'].apply(lambda string: replace_matches(string, mention_str, ' ')).apply(lambda y: np.nan if len(y)==0 else y)

# replace urls with the text 'URL'
df_train['cleantext'] = df_train['cleantext'].apply(lambda string: replace_matches(string, url_regex, 'URL')).apply(lambda y: np.nan if len(y)==0 else y)

# replace special characters
df_train['cleantext'] = df_train['cleantext'].apply(lambda string: replace_matches(string, special_chars, ' ')).apply(lambda y: np.nan if len(y)==0 else y)
df_train[['text','cleantext']].sample(50)

,text,cleantext
6667,Purple Heart Vet Finds Jihad Threat on His Car...,Purple Heart Vet Finds Jihad Threat on His Car...
1078,@dopeitsval ahh you're bomb baby ??,ahh you re bomb baby
940,Vanessa's game has officially blown up. LADIES...,Vanessa s game has officially blown up LADIES...
5858,Damn Wale knows how to ruin a song??,Damn Wale knows how to ruin a song
6981,950. If a landslide tumbles down todayI'm on y...,If a landslide tumbles down todayI m on y...
4862,Reminder: Mass murderer and white supremacist ...,Reminder Mass murderer and white supremacist ...
5884,Watch This Airport Get Swallowed Up By A Sands...,Watch This Airport Get Swallowed Up By A Sands...
4552,@ChristieC733 please support this Please cospo...,please support this Please cosponsor S ...
2210,China is only delaying the deluge: If the fund...,China is only delaying the deluge If the fund...
7050,I think a Typhoon just passed through here lol,I think a Typhoon just passed through here lol


For using the meaning of the words the upper/lower case is preventing proper word frequencies, so we'll lower the text as a final preprocessing step, and formalize the approach in order to be able to repeat it easily for the test set.

In [16]:
mention_str = '@([^ ]+)'
special_chars = '[^a-zA-Z]'
url_regex = '(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])'

def replace_matches(sentence,expr,replacement) :
    
    return re.sub(expr,replacement,sentence)

def cleaning_pipeline(sentence) :
    without_mentions = replace_matches(sentence, mention_str, ' ')
    without_urls = replace_matches(without_mentions, url_regex, 'URL')
    without_specials = replace_matches(without_urls, special_chars, ' ')
    return without_specials.lower()


# remove mentions
df_train['cleantext'] = df_train['text'].apply(lambda string: cleaning_pipeline(string))
df_train[['text','cleantext']].sample(50)

,text,cleantext
3266,He came to a land which was engulfed in tribal...,he came to a land which was engulfed in tribal...
2881,@_gaabyx we got purple activist I thought it w...,we got purple activist i thought it was a dr...
6475,Yup. Still hasn't sunk in. ?? https://t.co/Ii2...,yup still hasn t sunk in url
7025,@FoxNews let me report it to u people instead ...,let me report it to u people instead mr obam...
4239,#foodscare #offers2go #NestleIndia slips into ...,foodscare offers go nestleindia slips into ...
1687,and even if the stars and moon collide ÛÓ oh ...,and even if the stars and moon collide oh ...
1120,@antpips67 @JohnEJefferson obviously I'm aware...,obviously i m aware that not all as are fr...
6753,I liked a @YouTube video http://t.co/itnKBxgWL...,i liked a video url lexi belle for oklahoma ...
1646,I just collapsed in my bed ugh I'm exhausted,i just collapsed in my bed ugh i m exhausted
4359,Remove the http://t.co/Xxj2B4JxRt and Linkury ...,remove the url and linkury browser hijacker ur...


# Transforming the text data to vectors

With the cleaned but raw text we can't train a classification model, especially not a deep learning model since this requires vectorized input data. So how can we vectorize the text data? There are a few approaches
- Word frequency based (e.g. TF-IDF)
- Semantic meaning based (e.g. word2vec)
- Positional meaning (e.g. with RNN)


## Term Frequency–Inverse Document Frequency (TF-IDF)
One simple way of vectorizing text data is to construct new columns for every new word that is encountered, and counting the number of times a word is present in part of the text. This basic idea has a few caveats. Consider the word 'the' for instance, it does not have much meaning since it is often present in text. This problem can be overcome by using TF-IDF instead. TF-IDF is a combination of Term Frequency and Inverse Term Frequency. The term frequency is the frequency of a term in a document. The inverse term frequency is the frequency a term is present in a (large) collection of documents. If we divide the term frequency in a single document by the term frequency in a collection of a document we can capture the terms in a document that stand out from terms from a collection of documents. A high weight in TF-IDF is reached by a high term frequency (in the given document) and a low document frequency of the term in the whole collection of documents; the weights hence tend to filter out common terms.

## Word2Vec
Although TF-IDF is a nice way to vectorize text it doesn't capture the meaning of terms in a context. Therefore word2vec is proposed as a model to improve capturing the meaning of words from the context. Word2Vec is trained by taking out words of a sentence, replace it with a proposed word, and compare it with the actual word. This way, either the context can be learned around words (skip-gram) or the word in a context (Continuous Bag Of Words).

## Recurrent Neural Networks
Another approach is to use Recurrent Neural Networks. The idea is that recurrent neural networks would be able to capture the meaning of the sentence better by taking the order of words into account. Training of RNNs is done by taking the state from previous hidden layers into account. The values learned in the hidden layers from the first run become part of the input to the same hidden layers in the second run. Similarly, the values learned in the hidden layer on the second run become part of the input to the same hidden layer in the third run. In this way, the recurrent neural network gradually trains and predicts the meaning of the entire sequence rather than just the meaning of individual words.

We will compare the three approaches on the cleaned input text.

- For the TF-IDF approach we will use the TF-IDF transformer from scikit-learn, followed by a classifier
- For the Word2Vec approach we will use the implementation from gensim, followed by a classifier
- For the RNN we will design a network with Keras

Let's start!

In [17]:
# Split train data set in train and validation set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train['cleantext'], 
                                                    df_train['target'], 
                                                    random_state=0)

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data
vectorizer = TfidfVectorizer()
X_train_transformed = vectorizer.fit_transform(X_train)    
feature_names = np.array(vectorizer.get_feature_names())

# sort the TF-IDF m
sorted_tfidf_index = X_train_transformed.max(0).toarray()[0].argsort()

Out of curiosity, let's check the lowest and highest TF-IDF.

In [28]:
smallest_tfidf = pd.Series(sorted_tfidf_index[:20],index=feature_names[sorted_tfidf_index[:20]]).sort_values(ascending=True)
smallest_tfidf

comin          2042
countynews     2279
doublecups     3013
elephants      3280
gpv            4418
ika            5102
iturasya       5430
itz            5431
knw            5770
nowwwwww       7251
nuh            7269
prod           8182
recip          8521
rite           8844
rubbery        8970
thereal       10558
tlk           10690
vid           11319
widout        11639
wnw           11709
dtype: int64

In [29]:
largest_tfidf = pd.Series(sorted_tfidf_index[:-21:-1],index=feature_names[sorted_tfidf_index[:-21:-1]]).sort_values(ascending=False)
largest_tfidf

traumatised    10851
thunder        10635
thanks         10523
thank          10518
survive        10249
screams         9184
screaming       9183
rioting         8829
obliterated     7307
meltdown        6543
looooool        6185
ight            5084
fatality        3719
err             3441
devastated      2766
destroyed       2746
crushed         2401
craving         2329
cooool          2212
armageddon       521
dtype: int64

In [34]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = MultinomialNB(alpha=0.1)
clf.fit(X_train_transformed, y_train)
    
y_pred = clf.predict(vectorizer.transform(X_test))
accuracy_score(y_test,y_pred)

0.7962184873949579

Ok, this performance is OK, but not outstanding. We could tweak the parameters a bit and check other classifiers, but let's move on to Word2Vec. We will Use the Pre-trained word2Vec model from google, It contains word vectors for a vocabulary of 3 million words. It is trained on around 100 billion words from the google news dataset.
Download link: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing

In [37]:
import gensim
#load pretrained model from Google
GoogleModel = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True,)



In [51]:
# Count vectorization of text
from sklearn.feature_extraction.text import CountVectorizer

# Ticket Data
corpus = X_train

# Creating the vectorizer
vectorizer = CountVectorizer(stop_words='english')

# Fit the vectorizer
X = vectorizer.fit_transform(corpus)

CountVectorizedData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

# Creating the list of words which are present in the Document term matrix
WordsVocab=CountVectorizedData.columns[:]


In [52]:

# Defining a function which takes text input and returns one vector for each sentence
def FunctionText2Vec(inpTextData):
    # Converting the text to numeric data
    X = vectorizer.transform(inpTextData)
    CountVecData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
    
    # Creating empty dataframe to hold sentences
    W2Vec_Data=pd.DataFrame()
    
    # Looping through each row for the data
    for i in range(CountVecData.shape[0]):
 
        # initiating a sentence with all zeros
        Sentence = np.zeros(300)
 
        # Looping thru each word in the sentence and if its present in 
        # the Word2Vec model then storing its vector
        for word in WordsVocab[CountVecData.iloc[i,:]>=1]:
            #print(word)
            if word in GoogleModel.key_to_index.keys():    
                Sentence=Sentence+GoogleModel[word]
        # Appending the sentence to the dataframe
        W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
    return(W2Vec_Data)

# Calling the function to convert all the text data to Word2Vec Vectors
W2Vec_Data=FunctionText2Vec(X_train)
 
# Checking the new representation for sentences
W2Vec_Data.shape

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

...

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

(5709, 300)

In [53]:
from sklearn.preprocessing import MinMaxScaler

PredictorScaler=MinMaxScaler()
 
# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(W2Vec_Data)
 
# Generating the standardized values of X
X_train_scaled=PredictorScalerFit.transform(W2Vec_Data)
W2Vec_data_test = FunctionText2Vec(X_test)
X_test_scaled=PredictorScalerFit.transform(W2Vec_data_test)


/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

...

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  W2Vec_Data=W2Vec_Data.append(pd.DataFrame([Sentence]))
/var/folders/p7/6v1j479d7bn9mv_1s_mktx7r0000gn/T/ipykernel_2566/3274144664.py:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versi

In [54]:
from sklearn.svm import SVC

clf = SVC()
clf.fit(X_train_scaled, y_train)
    
y_pred = clf.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

0.8098739495798319

In [56]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X_train_scaled, y_train)
    
y_pred = clf.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

0.7358193277310925

In [57]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=10, random_state=0,n_estimators=120)
clf.fit(X_train_scaled, y_train)
    
y_pred = clf.predict(X_test_scaled)
accuracy_score(y_test,y_pred)

0.7925420168067226

As we can see the Word2Vec preparation did not lead to much better accuracy results. Hopefully the application of RNN will lead to a big step forward.
# RNN

First define the text encoder. Then define a basic RNN model with a Bidirectional LSTM in order to capture context around words (and not only the preceding words). Use BinaryCrossentropy, since this is a binary classification problem. 

In [64]:
import tensorflow as tf

VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(X_train.to_numpy())
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'url', 'the', 'a', 'in', 'to', 'of', 'i', 'and', 's',
       'is', 'for', 'you', 'on', 'it', 'my', 'that', 'with', 'at'],
      dtype='<U13')

In [65]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])


In [67]:
history = model.fit(X_train,y_train, epochs=10,
                    validation_data=(X_test.to_numpy(),y_test.to_numpy()),
                    validation_steps=30)

Epoch 1/10
179/179 [==============================] - 9s 23ms/step - loss: 0.6819 - accuracy: 0.5666 - val_loss: 0.6591 - val_accuracy: 0.5814
Epoch 2/10
179/179 [==============================] - 2s 14ms/step - loss: 0.5999 - accuracy: 0.6197 - val_loss: 0.5226 - val_accuracy: 0.7516
Epoch 3/10
179/179 [==============================] - 3s 14ms/step - loss: 0.4784 - accuracy: 0.7739 - val_loss: 0.4770 - val_accuracy: 0.7773
Epoch 4/10
179/179 [==============================] - 3s 15ms/step - loss: 0.4261 - accuracy: 0.8063 - val_loss: 0.4620 - val_accuracy: 0.7799
Epoch 5/10
179/179 [==============================] - 3s 14ms/step - loss: 0.3997 - accuracy: 0.8236 - val_loss: 0.4534 - val_accuracy: 0.7857
Epoch 6/10
179/179 [==============================] - 3s 14ms/step - loss: 0.3810 - accuracy: 0.8301 - val_loss: 0.4601 - val_accuracy: 0.8009
Epoch 7/10
179/179 [==============================] - 3s 14ms/step - loss: 0.3678 - accuracy: 0.8374 - val_loss: 0.4611 - val_accuracy: 0.7967

Let's see if changing the learning rate does have a positive impact.

In [68]:
model2 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-5),
              metrics=['accuracy'])
history = model2.fit(X_train,y_train, epochs=10,
                    validation_data=(X_test.to_numpy(),y_test.to_numpy()),
                    validation_steps=30)


Epoch 1/10
179/179 [==============================] - 9s 25ms/step - loss: 0.6928 - accuracy: 0.5666 - val_loss: 0.6919 - val_accuracy: 0.5814
Epoch 2/10
179/179 [==============================] - 3s 14ms/step - loss: 0.6914 - accuracy: 0.5666 - val_loss: 0.6900 - val_accuracy: 0.5814
Epoch 3/10
179/179 [==============================] - 3s 14ms/step - loss: 0.6896 - accuracy: 0.5666 - val_loss: 0.6878 - val_accuracy: 0.5814
Epoch 4/10
179/179 [==============================] - 3s 15ms/step - loss: 0.6875 - accuracy: 0.5666 - val_loss: 0.6850 - val_accuracy: 0.5814
Epoch 5/10
179/179 [==============================] - 3s 14ms/step - loss: 0.6851 - accuracy: 0.5666 - val_loss: 0.6820 - val_accuracy: 0.5814
Epoch 6/10
179/179 [==============================] - 3s 14ms/step - loss: 0.6823 - accuracy: 0.5666 - val_loss: 0.6783 - val_accuracy: 0.5814
Epoch 7/10
179/179 [==============================] - 3s 14ms/step - loss: 0.6789 - accuracy: 0.5666 - val_loss: 0.6741 - val_accuracy: 0.5814

Check if setting the activation function to sigmoid has a positive effect.

In [69]:
model3 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='sigmoid'),
    tf.keras.layers.Dense(1)
])

model3.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
history = model3.fit(X_train,y_train, epochs=10,
                    validation_data=(X_test.to_numpy(),y_test.to_numpy()),
                    validation_steps=30)



Epoch 1/10
179/179 [==============================] - 9s 26ms/step - loss: 0.6917 - accuracy: 0.5666 - val_loss: 0.6699 - val_accuracy: 0.5814
Epoch 2/10
179/179 [==============================] - 3s 14ms/step - loss: 0.6324 - accuracy: 0.5899 - val_loss: 0.5645 - val_accuracy: 0.7048
Epoch 3/10
179/179 [==============================] - 3s 14ms/step - loss: 0.5088 - accuracy: 0.7674 - val_loss: 0.5037 - val_accuracy: 0.7742
Epoch 4/10
179/179 [==============================] - 3s 14ms/step - loss: 0.4494 - accuracy: 0.7989 - val_loss: 0.4829 - val_accuracy: 0.7784
Epoch 5/10
179/179 [==============================] - 3s 15ms/step - loss: 0.4215 - accuracy: 0.8133 - val_loss: 0.4736 - val_accuracy: 0.7883
Epoch 6/10
179/179 [==============================] - 3s 14ms/step - loss: 0.4029 - accuracy: 0.8243 - val_loss: 0.4694 - val_accuracy: 0.7910
Epoch 7/10
179/179 [==============================] - 3s 15ms/step - loss: 0.3876 - accuracy: 0.8303 - val_loss: 0.4700 - val_accuracy: 0.7910

The effect is neglectable, so check if changing the size of the LSTM layer has positive impact.

In [81]:
model4 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=120,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(120)),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(1)
])

model4.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
history = model4.fit(X_train,y_train, epochs=5,
                    validation_data=(X_test.to_numpy(),y_test.to_numpy()),
                    validation_steps=30)



Epoch 1/5
179/179 [==============================] - 11s 31ms/step - loss: 0.6752 - accuracy: 0.5672 - val_loss: 0.6167 - val_accuracy: 0.6029
Epoch 2/5
179/179 [==============================] - 4s 22ms/step - loss: 0.5261 - accuracy: 0.7441 - val_loss: 0.4908 - val_accuracy: 0.7600
Epoch 3/5
179/179 [==============================] - 4s 22ms/step - loss: 0.4429 - accuracy: 0.7961 - val_loss: 0.4651 - val_accuracy: 0.7920
Epoch 4/5
179/179 [==============================] - 4s 22ms/step - loss: 0.4051 - accuracy: 0.8157 - val_loss: 0.4570 - val_accuracy: 0.7999
Epoch 5/5
179/179 [==============================] - 4s 24ms/step - loss: 0.3788 - accuracy: 0.8329 - val_loss: 0.4614 - val_accuracy: 0.8020


Evaluate if changing from LSTM to GRU has better results.

In [88]:
model5 = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=36,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(36)),
    tf.keras.layers.Dense(36, activation='relu'),
    tf.keras.layers.Dense(1)
])

model5.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(0.00015),
              metrics=['accuracy'])
history = model5.fit(X_train,y_train, epochs=8,
                    validation_data=(X_test.to_numpy(),y_test.to_numpy()),
                    validation_steps=30)



Epoch 1/8
179/179 [==============================] - 8s 21ms/step - loss: 0.6828 - accuracy: 0.5666 - val_loss: 0.6662 - val_accuracy: 0.5814
Epoch 2/8
179/179 [==============================] - 2s 12ms/step - loss: 0.6358 - accuracy: 0.5751 - val_loss: 0.5746 - val_accuracy: 0.6728
Epoch 3/8
179/179 [==============================] - 2s 12ms/step - loss: 0.4950 - accuracy: 0.7609 - val_loss: 0.4759 - val_accuracy: 0.7889
Epoch 4/8
179/179 [==============================] - 2s 12ms/step - loss: 0.4187 - accuracy: 0.8080 - val_loss: 0.4618 - val_accuracy: 0.7799
Epoch 5/8
179/179 [==============================] - 2s 12ms/step - loss: 0.3889 - accuracy: 0.8219 - val_loss: 0.4521 - val_accuracy: 0.7973
Epoch 6/8
179/179 [==============================] - 2s 12ms/step - loss: 0.3702 - accuracy: 0.8346 - val_loss: 0.4546 - val_accuracy: 0.7978
Epoch 7/8
179/179 [==============================] - 2s 12ms/step - loss: 0.3591 - accuracy: 0.8385 - val_loss: 0.4634 - val_accuracy: 0.7952
Epoch 

Best model was model4, so we will use this for the predictions. It seems the performance would be around 0.80. Due to the relatively small size of the input data, the preprocessing steps seems to have the largest impact on the performance of the models. This would be the stage where I would continue working on to improve my position in the competition.

# Predictions

Run the model on the test data after performing the same cleaning steps as during training.

In [94]:

df_test['cleantext'] = df_test['text'].apply(lambda string: cleaning_pipeline(string))

result = model5.predict(df_test['cleantext'].to_numpy())
result[result>=0.5]=1
result[result<0.5]=0
sample_submission=pd.read_csv('input/sample_submission.csv')
sample_submission['target']=result
sample_submission['target']=sample_submission['target'].astype(int)
#sample_submission.head()
sample_submission.to_csv('submission.csv', index=False)

102/102 [==============================] - 0s 2ms/step
